In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(38987, 128, 128, 3)

In [8]:
y.shape

(38987, 21)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(31189, 128, 128, 3)

In [11]:
X_test.shape

(7798, 128, 128, 3)

In [12]:
y_train.shape

(31189, 21)

In [13]:
y_test.shape

(7798, 21)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

21

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-06-01 20:27:17.969333: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


975/975 [==============================] - 254s 260ms/step - loss: 1.0319 - accuracy: 0.6683 - val_loss: 0.5304 - val_accuracy: 0.8194
Epoch 2/10
975/975 [==============================] - 257s 264ms/step - loss: 0.4181 - accuracy: 0.8617 - val_loss: 0.3249 - val_accuracy: 0.8878
Epoch 3/10
975/975 [==============================] - 267s 273ms/step - loss: 0.2801 - accuracy: 0.9079 - val_loss: 0.2897 - val_accuracy: 0.9013
Epoch 4/10
975/975 [==============================] - 270s 277ms/step - loss: 0.2131 - accuracy: 0.9292 - val_loss: 0.2981 - val_accuracy: 0.8957
Epoch 5/10
975/975 [==============================] - 259s 266ms/step - loss: 0.1805 - accuracy: 0.9395 - val_loss: 0.1387 - val_accuracy: 0.9514
Epoch 6/10
975/975 [==============================] - 241s 247ms/step - loss: 0.1454 - accuracy: 0.9505 - val_loss: 0.1798 - val_accuracy: 0.9392
Epoch 7/10
975/975 [==============================] - 226s 232ms/step - loss: 0.1290 - accuracy: 0.9561 - val_loss: 0.1485 - val_accura

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 91.37%


In [19]:
cnn_scores

[0.2918504774570465, 0.9136958122253418]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

244/244 [==============================] - 14s 58ms/step
Accuracy: 91.37%
Precision: 92.93%
Recall: 91.37%
F1 score: 91.45%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-7-jenis-daun.csv', index=False)

+------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                      | True Labels                                        | Predicted Labels                                   |
|------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|    0 | 24ff85f7-a5d3-4a55-92dd-e852e19fa4ec___Rutg._Bact.S 2241.JPG                  | Peach___Bacterial_spot                             | Peach___Bacterial_spot                             |
|    1 | ccf85694-b813-4442-bc83-3ff017fc7719___FAM_L.Blight 4825.JPG                  | Grape___Leaf_blight_(Isariopsis_Leaf_Spot)         | Grape___Leaf_blight_(Isariopsis_Leaf_Spot)         |
|    2 | ccf85694-b813-44

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-7-jenis-daun.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                   | True Labels                                        | Predicted Labels                                   |
|------+----------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|   28 | 474fb785-a088-4f76-8874-d7a0893ebb64___Rutg._HL 2414.JPG                   | Peach___healthy                                    | Apple___healthy                                    |
|   49 | 3fe25d8e-4497-49bd-b4e0-9c7dd3c3c6cd___FREC_C.Rust 9960_new30degFlipLR.JPG | Apple___Cedar_apple_rust                           | Apple___Apple_scab                                 |
|   57 | 853c3d19-728

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

975/975 [==============================] - 59s 60ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 9309.783993959427 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

244/244 [==============================] - 11s 46ms/step
Random Forest Classifier Accuracy: 97.32%
Random Forest Classifier Precision: 0.97
Random Forest Classifier Recall: 0.97
Random Forest Classifier F1 Score: 0.97


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-7-jenis-daun.csv', index=False)

+------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                      | True Labels                                        | Predicted Labels                                   |
|------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|    0 | 24ff85f7-a5d3-4a55-92dd-e852e19fa4ec___Rutg._Bact.S 2241.JPG                  | Peach___Bacterial_spot                             | Peach___Bacterial_spot                             |
|    1 | ccf85694-b813-4442-bc83-3ff017fc7719___FAM_L.Blight 4825.JPG                  | Grape___Leaf_blight_(Isariopsis_Leaf_Spot)         | Grape___Leaf_blight_(Isariopsis_Leaf_Spot)         |
|    2 | ccf85694-b813-44

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-7-jenis-daun.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                   | True Labels                                        | Predicted Labels                                   |
|------+----------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|   20 | 1bc60205-27cc-4d10-bc87-7e968573e5d5___FREC_Scab 3531_90deg.JPG            | Apple___Apple_scab                                 | Apple___Black_rot                                  |
|   59 | ed151c2d-14c0-49e0-bab4-48e5d879d9ca___RS_HL 7554_flipTB.JPG               | Apple___healthy                                    | Potato___healthy                                   |
|  113 | 2a510a2e-0e1